In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('..', 'scripts'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
%matplotlib notebook
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import ipywidgets as widgets

import altair as alt
import pandas as pd
from datetime import datetime

In [ ]:
from pointcloud_db import ConePositionTable, PointCloudTable, get_run_progressive_bounding_boxes

In [ ]:
hist = get_run_progressive_bounding_boxes(position_step=10, delta=20)

In [ ]:
time_zero = datetime.strptime(hist[0][2], '%Y-%m-%d %H:%M:%S.%f')
for i in range(len(hist)):
    hist[i] += (datetime.strptime(hist[i][2], '%Y-%m-%d %H:%M:%S.%f'), )

In [ ]:
df = pd.DataFrame(hist, columns=['size', 'dist', 'time_str', 'time'])

bars = alt.Chart(df).mark_bar().encode(
    x=alt.X("dist:Q", bin=True),
    y='mean(size):Q',
).properties(
    height=200,
    width=800,
)

heatmap = alt.Chart(df).mark_rect().encode(
    alt.X('dist:Q', bin=alt.Bin(maxbins=8)),
    alt.Color('count():Q', scale=alt.Scale(scheme='greenblue'))
).properties(
    width=800,
)

bubble = alt.Chart(df).mark_circle().encode(
    x='dist:Q',
    y='time:T',
    size='size:Q'
).properties(
    width=800,
)

hist_plot = alt.vconcat(bars, heatmap, bubble)

In [ ]:
hist_plot